In [1]:
import pyspark.sql.functions as F

import numpy as np
import pandas as pd
from geopy import distance

import hdbscan

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
ais = spark.table('af_vault.ais')
print(ais.count())
ais = ais.filter(ais['basedatetime']!='BaseDateTime')
print(ais.count())
# ais = ais.filter(ais['imo']!='')
print(ais.count())
ais.cache()

9088564
9088555
9088555


DataFrame[mmsi: string, basedatetime: string, lat: float, lon: float, sog: float, cog: float, heading: float, vesselname: string, imo: string, callsign: string, vesseltype: string, status: string, length: float, width: float, draft: float, cargo: int]

In [5]:
ais = ais.withColumn('year', ais['basedatetime'].substr(1,4))
ais = ais.withColumn('month', ais['basedatetime'].substr(6,2))
ais = ais.withColumn('day', ais['basedatetime'].substr(9,2))
ais = ais.withColumn('hour', ais['basedatetime'].substr(12,2))
ais = ais.withColumn('min', ais['basedatetime'].substr(15,2))
ais = ais.withColumn('sec', ais['basedatetime'].substr(18,2))

In [6]:
ais.show(4)

+---------+-------------------+--------+----------+----+------+-------+----------+----------+--------+----------+--------------------+------+-----+-----+-----+----+-----+---+----+---+---+
|     mmsi|       basedatetime|     lat|       lon| sog|   cog|heading|vesselname|       imo|callsign|vesseltype|              status|length|width|draft|cargo|year|month|day|hour|min|sec|
+---------+-------------------+--------+----------+----+------+-------+----------+----------+--------+----------+--------------------+------+-----+-----+-----+----+-----+---+----+---+---+
|235091871|2015-01-01T00:08:26|52.78763|-175.62761|10.3|  74.5|   86.0|EVA BULKER|IMO9544164|   2FJU4|        70|under way using e...| 185.0| 31.0|  6.6|   70|2015|   01| 01|  00| 08| 26|
|247119100|2015-01-01T05:36:17|52.87994|-176.21738|10.7|-148.8|  263.0|      POLE|IMO9128245|    IBTE|        70|under way using e...| 224.0| 32.0|-12.8|   70|2015|   01| 01|  05| 36| 17|
|247119100|2015-01-01T06:28:57|52.83234|-176.46661|11.0|-160

In [7]:
ais.filter((ais['year']=='2015') & (ais['month']=='01') & (ais['day']=='01')).count()

81434

In [8]:
sdf = ais.filter((ais['year']=='2015') & (ais['month']=='01') & (ais['day']=='01'))
df = sdf.toPandas()

In [10]:
df.head()

,mmsi,basedatetime,lat,lon,sog,cog,heading,vesselname,imo,callsign,...,length,width,draft,cargo,year,month,day,hour,min,sec
0,235091871,2015-01-01T00:08:26,52.787628,-175.627609,10.3,74.500000,86.0,EVA BULKER,IMO9544164,2FJU4,...,185.0,31.0,6.6,70.0,2015,01,01,00,08,26
1,247119100,2015-01-01T05:36:17,52.879940,-176.217377,10.7,-148.800003,263.0,POLE,IMO9128245,IBTE,...,224.0,32.0,-12.8,70.0,2015,01,01,05,36,17
2,247119100,2015-01-01T06:28:57,52.832340,-176.466614,11.0,-160.800003,254.0,POLE,IMO9128245,IBTE,...,224.0,32.0,-12.8,70.0,2015,01,01,06,28,57
3,247119100,2015-01-01T06:32:27,52.828510,-176.482910,11.0,-160.600006,254.0,POLE,IMO9128245,IBTE,...,224.0,32.0,-12.8,70.0,2015,01,01,06,32,27
4,247119100,2015-01-01T06:36:07,52.824459,-176.500214,11.0,-160.000000,254.0,POLE,IMO9128245,IBTE,...,224.0,32.0,-12.8,70.0,2015,01,01,06,36,07


In [11]:
df.groupby('imo').count().sort_values('mmsi')

,mmsi,basedatetime,lat,lon,sog,cog,heading,vesselname,callsign,vesseltype,...,length,width,draft,cargo,year,month,day,hour,min,sec
imo,,,,,,,,,,,,,,,,,,,,,
IMO9221097,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
IMO7042851,14,14,14,14,14,14,14,14,14,14,...,14,14,14,0,14,14,14,14,14,14
IMO9514315,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
IMO8660129,22,22,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,22,22
IMO9700263,27,27,27,27,27,27,27,27,27,27,...,27,27,27,0,27,27,27,27,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMO8030726,1263,1263,1263,1263,1263,1263,1263,1263,1263,1263,...,1263,1263,1263,0,1263,1263,1263,1263,1263,1263
IMO8707616,1264,1264,1264,1264,1264,1264,1264,1264,1264,1264,...,1264,1264,1264,0,1264,1264,1264,1264,1264,1264
IMO7716816,1299,1299,1299,1299,1299,1299,1299,1299,1299,1299,...,1299,1299,1299,1299,1299,1299,1299,1299,1299,1299


### Naive clustering all points no aggregation

In [13]:
hdb = hdbscan.HDBSCAN()

In [14]:
hdb.fit(df[['lat','lon']])

HDBSCAN()

In [20]:
np.unique(hdb.labels_).shape

(3119,)

In [21]:
pd.Series(hdb.labels_).value_counts()

-1       11506
 83        993
 30        625
 742       483
 500       429
         ...  
 869         5
 480         5
 411         5
 560         5
 2311        5
Length: 3119, dtype: int64

In [22]:
df['cluster'] = hdb.labels_

In [24]:
df.groupby(['cluster'])[['vesselname','imo','mmsi']].agg(pd.Series.nunique).sort_values('imo', ascending=False)

,vesselname,imo,mmsi
cluster,,,
-1,101,90,101
83,19,19,19
59,14,14,14
1246,14,14,14
60,13,13,13
...,...,...,...
1157,1,1,1
1158,1,1,1
1159,1,1,1


In [25]:
df[df['cluster']==83].groupby(['imo','vesselname','mmsi']).count()

,,,basedatetime,lat,lon,sog,cog,heading,callsign,vesseltype,status,length,width,draft,cargo,year,month,day,hour,min,sec,cluster
imo,vesselname,mmsi,,,,,,,,,,,,,,,,,,,,
,,477813100,7,7,7,7,7,7,7,7,7,0,0,0,0,7,7,7,7,7,7,7
IMO9162265,WESTWOOD PACIFIC,211517000,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57
IMO9193305,SEOUL EXPRESS,211331640,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
IMO9217644,SPARTIA,239715000,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
IMO9221097,COSCO SHANGHAI,235067987,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
IMO9245756,CORNELIA MAERSK,220061000,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
IMO9267912,JASPER ARROW,311919000,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
IMO9283863,CAPRICORNUS LEADER,564902000,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71
IMO9290854,IRIS HALO,371281000,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117,117


In [26]:
df[df['cluster']==59].groupby(['imo','vesselname','mmsi']).count()

,,,basedatetime,lat,lon,sog,cog,heading,callsign,vesseltype,status,length,width,draft,cargo,year,month,day,hour,min,sec,cluster
imo,vesselname,mmsi,,,,,,,,,,,,,,,,,,,,
IMO9162265,WESTWOOD PACIFIC,211517000,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
IMO9193305,SEOUL EXPRESS,211331640,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
IMO9217644,SPARTIA,239715000,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
IMO9260902,AEGIALI,636011627,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
IMO9267912,JASPER ARROW,311919000,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
IMO9283863,CAPRICORNUS LEADER,564902000,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
IMO9367176,OOCL KUALA LUMPUR,566747000,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
IMO9374052,CAPE VANGUARD,432835000,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
IMO9398448,ZIM ANTWERP,636014220,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


### Time aggregate to average position each hour

In [31]:
dfg = df.groupby(['vesselname','imo','mmsi','hour']).agg({'lat': np.mean, 'lon': np.mean})
dfg = dfg.reset_index()

In [90]:
dfg.shape

(1972, 7)

In [32]:
dfg.head()

,vesselname,imo,mmsi,hour,lat,lon
0,,,477813100,00,54.255581,-164.129562
1,,,477813100,03,54.156178,-163.021622
2,,,477813100,04,54.125160,-162.666733
3,,,477813100,05,54.108070,-162.418747
4,,,970113086,20,53.845619,-166.579742


In [92]:
dfg[dfg['hour']=='01'].head()

,vesselname,imo,mmsi,hour,lat,lon,cluster
7,ADAMANT,IMO9684940,367565680,01,53.906086,-166.512177,c101
45,ALASKA JURIS,IMO7514854,368382000,01,53.843697,-166.580368,c-1
68,ALASKA MIST,IMO8836259,368583000,01,53.902920,-166.527649,c-1
98,ALASKA VICTORY,IMO7434042,338568000,01,53.845673,-166.579529,c89
122,ALASKA WARRIOR,IMO7716816,338250000,01,53.907703,-166.509735,c106


In [91]:
dfg[dfg['hour']=='01'].tail(10)

,vesselname,imo,mmsi,hour,lat,lon,cluster
1687,SEABROOKE,IMO7908237,367018620,01,53.877800,-166.549377,c62
1728,STANLEY K,,367406850,01,53.877689,-166.555603,c73
1751,STAR KIRKENES,IMO9396127,257310000,01,52.894253,-172.983124,c-1
1774,STORM.PETREL,,433867110,01,53.905312,-166.511597,c101
1798,SUNSET BAY,IMO7820150,338673000,01,53.891472,-166.534790,c-1
1824,U S INTREPID,IMO7802536,366761830,01,53.881008,-166.540207,c76
1848,U S LIBERATOR,IMO7926526,369283000,01,53.901081,-166.530533,c49
1872,VIKING.QUEEN,IMO7309729,367777000,01,53.869305,-166.554001,c88
1896,WESTERN MARINER,IMO7719648,366906730,01,53.852089,-166.572769,c-1
1949,ZONE FIVE,IMO8852239,368217000,01,55.058952,-162.325592,c13


In [37]:
df[df['vesselname'].map(lambda x: 'U S ' in x)]['vesselname'].value_counts()

U S INTREPID     1261
U S LIBERATOR    1260
Name: vesselname, dtype: int64

In [38]:
hdb = hdbscan.HDBSCAN()

In [40]:
hdb.fit(dfg[['lat','lon']])

HDBSCAN()

In [66]:
pd.Series(hdb.labels_).value_counts()

-1      270
 1       50
 27      33
 4       33
 47      27
       ... 
 43       6
 45       6
 107      6
 28       5
 73       5
Length: 110, dtype: int64

In [69]:
dfg['cluster'] = ['c' + str(x) for x in hdb.labels_]

In [70]:
dfg['cluster'].value_counts()

c-1    270
c1      50
c27     33
c4      33
c47     27
      ... 
c92      6
c45      6
c17      6
c28      5
c73      5
Name: cluster, Length: 110, dtype: int64

In [71]:
dfg[dfg['cluster']=='c27'].groupby(['vesselname','imo','mmsi','hour']).count()

lat  lon  cluster
vesselname          imo        mmsi      hour                   
AEGIALI             IMO9260902 636011627 16      1    1        1
                                         17      1    1        1
CAPE VANGUARD       IMO9374052 432835000 12      1    1        1
                                         13      1    1        1
CMB POMEROL         IMO9598335 373966000 19      1    1        1
                                         20      1    1        1
COSCO FOS           IMO9484302 477947700 06      1    1        1
                                         07      1    1        1
COSCO KOREA         IMO9448750 477685300 04      1    1        1
COSCO PRINCE RUPERT IMO9448803 477351400 12      1    1        1
                                         13      1    1        1
COSCO THAILAND      IMO9448798 477959200 04      1    1        1
                                         05      1    1        1
DOUBLE HARMONY      IMO9570400 352759000 03      1    1        1
                                         04      1    1        1
JASPER ARROW        IMO9267912 311919000 16      1    1        1
                                         17      1    1        1
KEVLEEN-K           IMO7049770 367151060 12      1    1        1
                                         13      1    1        1
                                         14      1    1        1
MALAKAND            IMO9304198 463041101 00      1    1        1
                                         01      1    1        1
MALOLO              IMO7614082 303295000 01      1    1        1
                                         02      1    1        1
                                         03      1    1        1
OOCL KUALA LUMPUR   IMO9367176 566747000 04      1    1        1
                                         05      1    1        1
SAMJOHN SOLIDARITY  IMO9434424 241018000 00      1    1        1
                                         01      1    1        1
                                         02      1    1        1
                                         03      1    1        1
WESTWOOD PACIFIC    IMO9162265 211517000 20      1    1        1
                                         21      1    1        1

In [74]:
dfg[dfg['cluster']=='c4'].groupby(['vesselname','imo','mmsi','hour']).count()

lat  lon  cluster
vesselname    imo        mmsi      hour                   
ALASKA SPIRIT IMO7933622 366542920 19      1    1        1
                                   20      1    1        1
                                   21      1    1        1
                                   22      1    1        1
BLUE PACIFIC  IMO6510746 366480000 00      1    1        1
                                   01      1    1        1
                                   02      1    1        1
                                   03      1    1        1
                                   04      1    1        1
                                   05      1    1        1
                                   06      1    1        1
                                   07      1    1        1
                                   08      1    1        1
                                   09      1    1        1
                                   10      1    1        1
                                   11      1    1        1
                                   12      1    1        1
                                   13      1    1        1
                                   14      1    1        1
                                   15      1    1        1
                                   16      1    1        1
                                   17      1    1        1
                                   18      1    1        1
                                   19      1    1        1
                                   20      1    1        1
                                   21      1    1        1
                                   22      1    1        1
                                   23      1    1        1
MALOLO        IMO7614082 303295000 16      1    1        1
                                   17      1    1        1
                                   18      1    1        1
                                   19      1    1        1
                                   20      1    1        1

In [73]:
dfg[dfg['vesselname']=='STAR KIRKENES']

,vesselname,imo,mmsi,hour,lat,lon,cluster
1751,STAR KIRKENES,IMO9396127,257310000,01,52.894253,-172.983124,c-1
1752,STAR KIRKENES,IMO9396127,257310000,02,52.865597,-173.079681,c-1
1753,STAR KIRKENES,IMO9396127,257310000,04,52.733078,-173.549622,c2
1754,STAR KIRKENES,IMO9396127,257310000,05,52.659340,-173.771286,c2
1755,STAR KIRKENES,IMO9396127,257310000,06,52.615105,-173.897873,c2
1756,STAR KIRKENES,IMO9396127,257310000,07,52.577629,-174.245148,c2
1757,STAR KIRKENES,IMO9396127,257310000,08,52.569687,-174.476669,c1
1758,STAR KIRKENES,IMO9396127,257310000,09,52.560043,-174.670364,c-1
1759,STAR KIRKENES,IMO9396127,257310000,10,52.550953,-174.918396,c-1
1760,STAR KIRKENES,IMO9396127,257310000,11,52.540180,-175.143143,c1


In [75]:
dfg[dfg['cluster']=='c1']

,vesselname,imo,mmsi,hour,lat,lon,cluster
824,E.R. DALLAS,IMO9282950,636090740,00,53.033752,-177.123093,c1
825,E.R. DALLAS,IMO9282950,636090740,01,52.998554,-177.403656,c1
826,E.R. DALLAS,IMO9282950,636090740,02,52.962482,-177.666275,c1
827,E.R. DALLAS,IMO9282950,636090740,03,52.928944,-177.886124,c1
828,E.R. DALLAS,IMO9282950,636090740,04,52.909176,-178.067535,c1
829,EVA BULKER,IMO9544164,235091871,00,52.787830,-175.626495,c1
944,GULF VALOUR,IMO9505821,311072100,07,53.180386,-176.865265,c1
945,GULF VALOUR,IMO9505821,311072100,08,53.108192,-177.043030,c1
946,GULF VALOUR,IMO9505821,311072100,09,52.955437,-177.393784,c1
947,GULF VALOUR,IMO9505821,311072100,10,52.845249,-177.653275,c1


In [49]:
import plotly.express as px

In [54]:
fig = px.scatter_geo(lat=dfg['lat'], lon=dfg['lon'], color=dfg['vesselname'], hover_name=dfg['hour'])
fig.show()

In [88]:
fig = px.scatter_geo(dfg, 
                     lat='lat', 
                     lon='lon', 
                     color='cluster', 
                     hover_name='hour', 
                     hover_data=['vesselname'],
                     color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

In [87]:
fig = px.scatter_geo(dfg, 
                     lat='lat', 
                     lon='lon', 
                     color='cluster', 
                     hover_name='hour', 
                     hover_data=['vesselname'],
                     color_discrete_sequence=px.colors.qualitative.Dark24, animation_frame="hour")
fig.show()

In [89]:
dfg.shape

(1972, 7)